In [ ]:
import sys
import os
import numpy as np
from PIL import Image
import datetime

import torch
import torch.nn as nn
from torch import optim

In [ ]:
def get_ids(directory):
    """Returns a list of the ids in the directory"""
    return (f[:-4] for f in os.listdir(directory))

In [ ]:
def get_ids(directory):
    """Returns a list of the ids in the directory"""
    return (f[:-4] for f in os.listdir(directory))

# In[255]:
#def load_img(ids, directory, suffix):
#    for iid in ids:
#        
#        img = Image.open(directory + iid + suffix)
#        img = img.resize((640,360))
#        img = np.array(img)
#        yield img
        
def load_img2(iid, directory, suffix):
        
    img = Image.open(directory + iid + suffix)
    img = img.resize((640,360))
#    img = img.resize((320,180))

    img = np.array(img)
        
    return img

# In[256]:
#def load_mask(ids, directory, suffix):
#    for iid in ids:
#        
#        msk = Image.open(directory + iid + "_drivable_id" + suffix)
#        msk = msk.resize((640,360))
#        msk = np.array(msk).astype('float32')
#        yield msk
        
def load_mask2(iid, directory, suffix):
        
    msk = Image.open(directory + iid + "_drivable_id" + suffix)
    msk = msk.resize((640,360))
#    msk = msk.resize((320,180))

    msk = np.array(msk).astype('float32')
    return msk

# In[257]:
def hwc_to_chw(img):
    return np.transpose(img, axes=[2, 0, 1])

# In[258]:
def normalize(x):
    return np.float32(x/255.)

In [ ]:
epochs = 100
batch_size = 1
lr = 0.0001
save_cp = False
gpu = False
load = True

In [ ]:
from unet import UNet_multiclass

net = UNet_multiclass(n_channels=3, n_classes=3)

if gpu: 
    net.cuda()
    
if load:
    weights = "checkpoints/driveable/BDD_3_CP19.pth"
    if torch.cuda.is_available():
        net.load_state_dict(torch.load(weights))
    else:
        net.load_state_dict(torch.load(weights, map_location='cpu'))
    print('Model loaded from {}'.format(weights))
    
net.eval()

In [ ]:
path = "/home/kenny/Desktop/bdd"
# path = "/home/datasets/bdd/bdd100k"

dir_img_train = path + '/images/100k/train/'
dir_img_val = path + '/images/100k/val/'

dir_mask_train = path + '/drivable_maps/labels/train/'
dir_mask_val = path + '/drivable_maps/labels/val/'

dir_checkpoint = 'checkpoints/driveable/'

ids_train = get_ids(dir_img_train)
ids_val = get_ids(dir_img_val)

iddataset = dict()

iddataset['train'] = list(ids_train)
iddataset['val'] = list(ids_val)

iddataset['train'] = iddataset['train'][:5]
iddataset['val'] = iddataset['val'][:5]

N_train = len(iddataset['train'])
N_val = len(iddataset['val'])

In [ ]:
N_train

In [ ]:
i = 3

fname = 'b2de6f59-9f74dea1'
img = load_img2(fname, dir_img_val, '.jpg')

# img = load_img2(iddataset['train'][i], dir_img_train, '.jpg')
img = hwc_to_chw(img)
img = normalize(img)
img = np.expand_dims(img, axis = 0)                

# msk = load_mask2(iddataset['train'][i], dir_mask_train, '.png')
msk = load_mask2(fname, dir_mask_val, '.png')

msk = np.expand_dims(msk, axis = 0)

img = torch.from_numpy(img)

if gpu:
    img = img.cuda()

In [ ]:
with torch.no_grad():
    mask_pred = net(img)

In [ ]:
y_pred = mask_pred.cpu().numpy()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# img = load_img2(iddataset['train'][i], dir_img_train, '.jpg')
img = load_img2(fname, dir_img_val, '.jpg')

plt.imshow(img)

In [ ]:
plt.imshow(np.argmax(y_pred[0], axis=0))

In [ ]:
plt.imshow(msk[0])

In [ ]:
import webcolors

hex_colors = ['#000000', '#0000FF', '#FF0000']

rgb_colors = []

for hex_color in hex_colors:
    rgb_colors.append(webcolors.hex_to_rgb(hex_color))
    
def colorize(mask, colors = np.array(rgb_colors)):
    colorMask = np.zeros([mask.shape[0], mask.shape[1], 3],dtype=np.uint8)
    for r in range(mask.shape[0]):
        for c in range(mask.shape[1]):
            colorMask[r,c,] = colors[mask[r,c]]

    return colorMask

In [ ]:
b = np.argmax(y_pred[0], axis=0)
# b = colorize(b)
gt = colorize(np.uint8(msk[0]))

In [ ]:
plt.imshow(img)
plt.imshow(b, alpha=0.25)

In [ ]:
plt.imshow(img)
plt.imshow(gt, alpha=0.25)

In [ ]:
show = img.copy()
for i in range(b.shape[0]):
    for j in range(b.shape[1]):
        if b[i,j] == 1:
            show[i,j] = (0,0,255)
        elif b[i,j] == 2:
            show[i,j] = (255,0,0)

In [ ]:
plt.imshow(show)

In [ ]:
show2 = Image.fromarray(show)
show2.save("bdd_val_pred_2", "PNG")

In [ ]:
show = img.copy()
for i in range(msk[0].shape[0]):
    for j in range(msk[0].shape[1]):
        if msk[0,i,j] == 1:
            show[i,j] = (0,0,255)
        elif msk[0,i,j] == 2:
            show[i,j] = (255,0,0)

In [ ]:
plt.imshow(show)

In [ ]:
from skimage import data, color, io, img_as_float

img_hsv = color.rgb2hsv(a)
color_mask_hsv = color.rgb2hsv(b)

In [ ]:
alpha = 0.5

img_hsv[..., 0] = color_mask_hsv[..., 0]
img_hsv[..., 1] = color_mask_hsv[..., 1] * alpha

img_masked = color.hsv2rgb(img_hsv)

plt.imshow(img_masked)

In [ ]:
plt.imshow(b)

In [ ]:
plt.imshow(np.ma.masked_array(img, mask=np.bool(b.any())))

In [ ]:
np.unique(e)

In [ ]:
e = b == 1

In [ ]:
e = np.dstack((e, e, e))

In [ ]:
e == True

In [ ]:
img2.shape

In [ ]:
plt.imshow(np.uint8(e))

In [ ]:
e.dtype

In [ ]:
e